# Projet 11 - Big Data Cloud : Classification de Fruits

**Auteur** : David Scanu  
**Date** : Octobre 2025  
**Objectif** : Développement local du pipeline PySpark avec broadcast TensorFlow et PCA

---

## 📋 Sommaire

1. [Setup et Configuration](#1-setup-et-configuration)
2. [Initialisation PySpark](#2-initialisation-pyspark)
3. [Chargement et Exploration des Données](#3-chargement-et-exploration-des-données)
4. [Extraction de Features avec TensorFlow](#4-extraction-de-features-avec-tensorflow)
5. [Broadcast des Poids du Modèle](#5-broadcast-des-poids-du-modèle)
6. [Réduction de Dimension avec PCA](#6-réduction-de-dimension-avec-pca)
7. [Tests et Validation](#7-tests-et-validation)

---

## ⚠️ Important

Ce notebook est conçu pour le **développement et test en local**.

- ✅ Tester et valider le code ici avant migration cloud
- ✅ Utiliser un subset des données pour itérer rapidement
- ❌ Ne PAS utiliser AWS EMR à ce stade (coûts)

Une fois validé localement, le code sera migré vers AWS EMR JupyterHub.

---
## 1. Setup et Configuration

### 1.1 Vérification de l'environnement

In [ ]:
# Vérifier les versions des packages
import sys
print(f"Python version: {sys.version}")

try:
    import pyspark
    print(f"PySpark version: {pyspark.__version__}")
except ImportError:
    print("⚠️ PySpark n'est pas installé")

try:
    import tensorflow as tf
    print(f"TensorFlow version: {tf.__version__}")
except ImportError:
    print("⚠️ TensorFlow n'est pas installé")

try:
    from PIL import Image
    print(f"PIL/Pillow: OK")
except ImportError:
    print("⚠️ Pillow n'est pas installé")

### 1.2 Installation des dépendances (si nécessaire)

**Note** : Décommenter et exécuter si les packages ne sont pas installés.

```bash
pip install pyspark==3.5.0 tensorflow==2.16.1 pillow pandas numpy pyarrow
```

**Prérequis** : Java JDK 11 ou 17 doit être installé pour PySpark.

### 1.3 Import des librairies

In [ ]:
# PySpark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, pandas_udf, element_at, split
from pyspark.sql.types import ArrayType, FloatType, StructType, StructField, StringType
from pyspark.ml.feature import PCA, VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT

# TensorFlow imports
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

# Autres imports
from PIL import Image
import pandas as pd
import numpy as np
import io
import os
from pathlib import Path

print("✅ Imports réussis")

### 1.4 Configuration des chemins

Adapter ces chemins selon votre environnement local.

In [ ]:
# Chemins locaux
PROJECT_ROOT = Path(os.getcwd()).parent
DATA_DIR = PROJECT_ROOT / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
FEATURES_DIR = DATA_DIR / "features"
PCA_DIR = DATA_DIR / "pca"

# Créer les dossiers s'ils n'existent pas
for directory in [DATA_DIR, RAW_DATA_DIR, FEATURES_DIR, PCA_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"📁 Dossier projet: {PROJECT_ROOT}")
print(f"📁 Dossier données: {DATA_DIR}")
print(f"📁 Données brutes: {RAW_DATA_DIR}")
print(f"📁 Features: {FEATURES_DIR}")
print(f"📁 PCA: {PCA_DIR}")

---
## 2. Initialisation PySpark

### 2.1 Création de la SparkSession

In [ ]:
# Configuration PySpark pour développement local
spark = SparkSession.builder \
    .appName("P11-Fruits-Local-Development") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.execution.arrow.maxRecordsPerBatch", "1024") \
    .getOrCreate()

# Configuration du niveau de log
spark.sparkContext.setLogLevel("WARN")

# Récupérer le SparkContext pour le broadcast
sc = spark.sparkContext

print(f"✅ SparkSession créée")
print(f"   Version Spark: {spark.version}")
print(f"   Master: {spark.sparkContext.master}")
print(f"   App Name: {spark.sparkContext.appName}")

---
## 3. Chargement et Exploration des Données

### 3.1 Téléchargement du dataset (si nécessaire)

**Option 1 - Téléchargement depuis Kaggle** :
```bash
# Installer Kaggle CLI si nécessaire
pip install kaggle

# Télécharger le dataset
kaggle datasets download -d moltean/fruits -p data/raw/ --unzip
```

**Option 2 - Téléchargement depuis le lien direct** :
```bash
# Télécharger et extraire
wget https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Data_Scientist_P8/fruits.zip -P data/raw/
unzip data/raw/fruits.zip -d data/raw/
```

**Option 3 - Subset pour tests rapides** :
Utiliser uniquement quelques classes pour tester le code rapidement.

### 3.2 Statistiques du dataset

Le dataset Fruits-360 contient deux versions:
- **fruits-360_dataset**: Images 100x100 pixels (utilisé ici)
- **fruits-360-original-size**: Images en tailles originales

**Contenu du dataset 100x100**:
- Training: 67,692 images, 131 classes
- Test: 22,688 images

Pour les tests locaux, il est recommandé de commencer avec un subset (par exemple les pommes) avant de traiter tout le dataset.

In [ ]:
# ✅ Dataset téléchargé et extrait dans data/raw/

# Structure du dataset:
# data/raw/
#   ├── fruits-360_dataset/fruits-360/
#   │   ├── Training/ (67,692 images, 131 classes)
#   │   └── Test/ (22,688 images)
#   └── fruits-360-original-size/ (tailles originales)

# Pour le développement, on utilise le dataset 100x100 (fruits-360_dataset)
IMAGES_PATH = str(RAW_DATA_DIR / "fruits-360_dataset" / "fruits-360" / "Training")
TEST_IMAGES_PATH = str(RAW_DATA_DIR / "fruits-360_dataset" / "fruits-360" / "Test")

print(f"📂 Chemin Training: {IMAGES_PATH}")
print(f"   Existe: {os.path.exists(IMAGES_PATH)}")
print(f"📂 Chemin Test: {TEST_IMAGES_PATH}")
print(f"   Existe: {os.path.exists(TEST_IMAGES_PATH)}")

# Compter les images et classes
if os.path.exists(IMAGES_PATH):
    classes = [d for d in os.listdir(IMAGES_PATH) if os.path.isdir(os.path.join(IMAGES_PATH, d))]
    print(f"\n📊 Statistiques:")
    print(f"   Nombre de classes: {len(classes)}")
    print(f"   Exemples de classes: {', '.join(classes[:5])}")

### 3.2 Chargement des images avec PySpark

Inspiré du code de l'alternant, nous allons charger les images en utilisant `binaryFiles`.

In [ ]:
# Charger les images depuis le système de fichiers local
# Pour AWS EMR, remplacer par: s3://bucket-name/path/to/images/*/*.jpg

# OPTION 1: Subset pour tests rapides (recommandé pour débuter)
# Charger seulement quelques classes de pommes pour tester rapidement
image_path = f"file://{IMAGES_PATH}/Apple*/*.jpg"
print(f"🔍 Mode: SUBSET (Apple* classes seulement)")

# OPTION 2: Toutes les images (67,692 images - peut être long ~15-30 min)
# Décommenter pour traiter tout le dataset:
# image_path = f"file://{IMAGES_PATH}/*/*.jpg"
# print(f"🔍 Mode: DATASET COMPLET (toutes les classes)")

print(f"🔍 Pattern de recherche: {image_path}")

# Charger les images avec PySpark
print("⏳ Chargement des images...")
df_images = spark.read.format("binaryFile").load(image_path)

print(f"✅ {df_images.count()} images chargées")
print(f"\nSchéma:")
df_images.printSchema()
print(f"\nAperçu:")
df_images.show(5, truncate=60)

### 3.3 Extraction du label depuis le chemin

Le nom de la classe (label) est dans le nom du dossier.

In [ ]:
# Extraire le label depuis le chemin du fichier
# Exemple: file:///path/to/Training/Apple Braeburn/image_001.jpg
# Label: Apple Braeburn

df_with_labels = df_images.withColumn(
    "label",
    element_at(split(col("path"), "/"), -2)
)

print(f"✅ Labels extraits")
print(f"\nAperçu des données:")
df_with_labels.select("path", "label").show(10, truncate=60)

# Compter les images par classe
print(f"\n📊 Distribution des classes:")
label_counts = df_with_labels.groupBy("label").count().orderBy("label")
label_counts.show(20, truncate=False)

---
## 4. Extraction de Features avec TensorFlow

### 4.1 Préparation du modèle MobileNetV2

Nous utilisons MobileNetV2 pré-entraîné sur ImageNet, sans la couche de classification (include_top=False).

In [ ]:
# Charger le modèle MobileNetV2 pour l'extraction de features
# include_top=False : on retire la couche de classification
# pooling='avg' : on applique un average pooling global
# Sortie: vecteur de 1280 features par image

model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    pooling='avg'
)

print("✅ Modèle MobileNetV2 chargé")
print(f"   Input shape: {model.input_shape}")
print(f"   Output shape: {model.output_shape}")
print(f"   Dimension des features: {model.output_shape[1]}")

# Afficher le résumé du modèle
model.summary()

---
## 5. Broadcast des Poids du Modèle

### 5.1 Pourquoi broadcaster les poids ?

**Problème** : Sans broadcast, chaque worker Spark recharge le modèle MobileNetV2 depuis internet ou disque, ce qui :
- Consomme beaucoup de bande passante
- Augmente le temps d'exécution
- Augmente la consommation mémoire

**Solution** : Utiliser `sc.broadcast()` pour distribuer les poids une seule fois à tous les workers.

### 5.2 Extraction et broadcast des poids

In [ ]:
# Extraire les poids du modèle
model_weights = model.get_weights()

print(f"📦 Nombre de tenseurs de poids: {len(model_weights)}")
print(f"📦 Taille approximative en mémoire: {sum([w.nbytes for w in model_weights]) / 1024 / 1024:.2f} MB")

# Broadcaster les poids à tous les workers
broadcast_weights = sc.broadcast(model_weights)

print("✅ Poids du modèle broadcastés")
print(f"   Broadcast variable ID: {broadcast_weights.id}")

### 5.3 Définition de la Pandas UDF avec broadcast

La Pandas UDF permet d'appliquer une fonction sur des batches de données de manière distribuée.

**Important** : Le modèle doit être reconstruit dans chaque worker avec les poids broadcastés.

In [ ]:
# Définir le schéma de sortie (array de 1280 floats)
features_schema = ArrayType(FloatType())

# Définir la Pandas UDF
@pandas_udf(features_schema)
def extract_features_udf(content_series: pd.Series) -> pd.Series:
    """
    Extrait les features d'images en utilisant MobileNetV2.
    
    Cette fonction est exécutée sur chaque worker Spark.
    Elle reconstruit le modèle avec les poids broadcastés.
    
    Args:
        content_series: Série Pandas contenant les données binaires des images
        
    Returns:
        Série Pandas contenant les features extraites (arrays de 1280 floats)
    """
    # Reconstruire le modèle dans le worker
    # weights=None car on va charger les poids broadcastés
    local_model = MobileNetV2(
        weights=None,
        include_top=False,
        pooling='avg'
    )
    
    # Charger les poids broadcastés
    local_model.set_weights(broadcast_weights.value)
    
    def process_image(content):
        """
        Traite une image individuelle.
        
        Args:
            content: Données binaires de l'image
            
        Returns:
            Array de features (1280 floats) ou None si erreur
        """
        try:
            # Charger l'image depuis les bytes
            img = Image.open(io.BytesIO(content))
            
            # Convertir en RGB si nécessaire
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Redimensionner à la taille attendue par MobileNetV2 (224x224)
            img = img.resize((224, 224))
            
            # Convertir en array numpy
            img_array = img_to_array(img)
            
            # Ajouter la dimension batch (1, 224, 224, 3)
            img_array = np.expand_dims(img_array, axis=0)
            
            # Prétraiter selon les attentes de MobileNetV2
            img_array = preprocess_input(img_array)
            
            # Extraire les features
            features = local_model.predict(img_array, verbose=0)
            
            # Retourner le vecteur de features (1280,)
            return features[0].tolist()
            
        except Exception as e:
            print(f"Erreur lors du traitement de l'image: {e}")
            return None
    
    # Appliquer le traitement sur toutes les images du batch
    return content_series.apply(process_image)

print("✅ Pandas UDF définie")
print("   Fonction: extract_features_udf")
print("   Utilise les poids broadcastés")

### 5.4 Application de l'extraction de features

**Note** : Cette étape peut prendre du temps selon le nombre d'images.

In [ ]:
# Appliquer l'extraction de features
# df_features = df_with_labels.withColumn(
#     "features",
#     extract_features_udf(col("content"))
# )

# # Filtrer les images où l'extraction a échoué
# df_features = df_features.filter(col("features").isNotNull())

# print(f"✅ Features extraites pour {df_features.count()} images")
# df_features.select("path", "label", "features").show(5, truncate=60)

print("⚠️ À exécuter une fois les images chargées")

### 5.5 Sauvegarde des features (optionnel)

In [ ]:
# Sauvegarder les features en format Parquet (efficace pour PySpark)
# features_output_path = str(FEATURES_DIR / "mobilenetv2_features")
# df_features.write.mode("overwrite").parquet(features_output_path)

# print(f"✅ Features sauvegardées: {features_output_path}")

print("⚠️ À exécuter une fois les features extraites")

---
## 6. Réduction de Dimension avec PCA

### 6.1 Préparation des données pour PCA

PySpark PCA attend un vecteur dense en entrée. Il faut donc transformer notre array de features.

In [ ]:
# Si on recharge depuis Parquet:
# df_features = spark.read.parquet(str(FEATURES_DIR / "mobilenetv2_features"))

# Convertir l'array de features en vecteur dense
# PySpark ML attend un VectorUDT

from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors, VectorUDT

# UDF pour convertir array -> vecteur dense
array_to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())

# df_for_pca = df_features.withColumn(
#     "features_vector",
#     array_to_vector(col("features"))
# )

# df_for_pca.select("label", "features_vector").show(5, truncate=60)

print("⚠️ À exécuter une fois les features extraites")

### 6.2 Application de la PCA

La PCA réduit la dimensionnalité de 1280 à k composantes principales.

**Choix de k** :
- k=100 : Réduction forte, moins d'information
- k=200 : Bon compromis
- k=500 : Réduction modérée, plus d'information

On peut aussi analyser la variance expliquée pour choisir k.

In [ ]:
# Nombre de composantes principales
K_COMPONENTS = 200

# Créer le modèle PCA
# pca = PCA(
#     k=K_COMPONENTS,
#     inputCol="features_vector",
#     outputCol="pca_features"
# )

# # Entraîner le modèle PCA
# pca_model = pca.fit(df_for_pca)

# # Appliquer la transformation PCA
# df_pca = pca_model.transform(df_for_pca)

# print(f"✅ PCA appliquée (réduction de 1280 à {K_COMPONENTS} dimensions)")
# df_pca.select("label", "pca_features").show(5, truncate=60)

print(f"⚠️ Configuration: k={K_COMPONENTS} composantes")
print("⚠️ À exécuter une fois les vecteurs de features préparés")

### 6.3 Analyse de la variance expliquée

In [ ]:
# Variance expliquée par chaque composante
# explained_variance = pca_model.explainedVariance

# print(f"📊 Variance expliquée:")
# print(f"   Total: {sum(explained_variance):.4f}")
# print(f"   Par composante (top 10):")
# for i, var in enumerate(explained_variance[:10]):
#     print(f"   PC{i+1}: {var:.6f}")

# # Variance cumulée
# import matplotlib.pyplot as plt
# cumsum_variance = np.cumsum(explained_variance)

# plt.figure(figsize=(10, 5))
# plt.plot(range(1, len(cumsum_variance) + 1), cumsum_variance)
# plt.xlabel('Nombre de composantes')
# plt.ylabel('Variance cumulée expliquée')
# plt.title('Variance expliquée par les composantes principales')
# plt.grid(True)
# plt.show()

print("⚠️ À exécuter une fois la PCA appliquée")

### 6.4 Sauvegarde des résultats PCA

In [ ]:
# Sélectionner les colonnes pertinentes
# df_final = df_pca.select("path", "label", "pca_features")

# # Sauvegarder en Parquet
# pca_output_path = str(PCA_DIR / "pca_results")
# df_final.write.mode("overwrite").parquet(pca_output_path)

# print(f"✅ Résultats PCA sauvegardés: {pca_output_path}")

# # Sauvegarder aussi en CSV pour inspection
# # Note: Le CSV sera partitionné en plusieurs fichiers
# csv_output_path = str(PCA_DIR / "pca_results_csv")
# df_final.write.mode("overwrite").option("header", "true").csv(csv_output_path)

# print(f"✅ Résultats PCA sauvegardés en CSV: {csv_output_path}")

print("⚠️ À exécuter une fois la PCA appliquée")

---
## 7. Tests et Validation

### 7.1 Vérification des dimensions

In [ ]:
# Vérifier les dimensions à chaque étape
# print("📊 Dimensions:")
# print(f"   Images originales: {df_images.count()} images")
# print(f"   Features extraites: {df_features.count()} images x 1280 features")
# print(f"   Après PCA: {df_pca.count()} images x {K_COMPONENTS} features")

print("⚠️ À exécuter pour vérifier le pipeline complet")

### 7.2 Test sur un échantillon

Tester le pipeline complet sur un petit échantillon pour valider.

In [ ]:
# # Prendre un échantillon de 100 images
# sample_images = df_images.limit(100)

# # Appliquer tout le pipeline
# sample_with_labels = sample_images.withColumn(
#     "label",
#     element_at(split(col("path"), "/"), -2)
# )

# sample_features = sample_with_labels.withColumn(
#     "features",
#     extract_features_udf(col("content"))
# ).filter(col("features").isNotNull())

# sample_vectors = sample_features.withColumn(
#     "features_vector",
#     array_to_vector(col("features"))
# )

# sample_pca = pca_model.transform(sample_vectors)

# print(f"✅ Test sur échantillon: {sample_pca.count()} images traitées")
# sample_pca.select("label", "pca_features").show(10, truncate=60)

print("⚠️ À exécuter pour tester le pipeline sur un échantillon")

---
## 8. Nettoyage et Arrêt

### 8.1 Libérer les ressources

In [ ]:
# Unpersist les DataFrames mis en cache (si utilisé .cache())
# df_features.unpersist()
# df_pca.unpersist()

# Détruire la variable broadcast
broadcast_weights.unpersist()

print("✅ Ressources libérées")

### 8.2 Arrêt de la SparkSession

In [ ]:
# Arrêter la SparkSession
# spark.stop()
# print("✅ SparkSession arrêtée")

print("⚠️ Décommenter pour arrêter Spark")

---
## 📝 Notes pour la migration vers AWS EMR

### Changements à apporter:

1. **Chemins S3**
   ```python
   # Local
   image_path = "file:///path/to/images/*/*.jpg"
   
   # AWS EMR
   image_path = "s3://mon-bucket-fruits/data/raw/Training/*/*.jpg"
   ```

2. **Configuration Spark**
   - Sur EMR JupyterHub, la SparkSession est déjà créée
   - Pas besoin de `.master("local[*]")`
   - Le SparkContext est accessible via `spark.sparkContext`

3. **Installation de packages**
   - TensorFlow doit être installé via bootstrap actions ou dans le notebook
   - Sur EMR 7.10.0: TensorFlow 2.16.1 peut être pré-installé

4. **Sauvegarde des résultats**
   ```python
   # Sauvegarder directement sur S3
   df_pca.write.mode("overwrite").csv("s3://mon-bucket-fruits/data/pca/")
   ```

5. **Monitoring**
   - Utiliser Spark UI pour suivre l'exécution
   - Accessible via le tunnel SSH et FoxyProxy

### Checklist avant migration:

- [ ] Code validé en local sur un subset
- [ ] Broadcast des poids fonctionne correctement
- [ ] PCA appliquée et validée
- [ ] Chemins adaptés pour S3
- [ ] Dataset uploadé sur S3
- [ ] Cluster EMR créé et configuré
- [ ] Tunnel SSH et FoxyProxy configurés
- [ ] Accès JupyterHub vérifié